### SVM Support Vector Machines

En que consiste support vector machine?
Por que es uno de los mejores modelos de machine learning?

Minimizar

${\displaystyle \left[{\frac {1}{n}}\sum _{i=1}^{n}\max \left(0,1-y_{i}({\vec {w}}\cdot {\vec {x}}_{i}-b)\right)\right]+\lambda \lVert {\vec {w}}\rVert ^{2},}$



Tomado de  wikipedia. https://en.wikipedia.org/wiki/Support-vector_machine
<img src="img/SVM_margin.png">

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
import numpy as np
import pandas as pd
from bokeh.palettes import Category10,Spectral6
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure
from bokeh.transform import factor_cmap
from bokeh.io import output_notebook, show

%matplotlib inline
output_notebook()
style.use('ggplot')

Loading BokehJS ...

In [105]:
n = 100
g1 = np.random.normal(0.5,1,size = (n,2))
y1 = [1 for i in range(n)]
g2 = np.random.normal(5,1,size = (n,2))
y2 = [-1 for i in range(n)]
y = np.concatenate((y1,y2),axis=0).reshape(-1,1)
X = np.concatenate((g1,g2),axis=0)
data_x =  np.concatenate((X , y ),axis=1)             
data =  pd.DataFrame(data_x,columns=["Nota1","Nota2","Label"])
data['color'] = data.Label.apply(lambda x: 'blue' if x==1 else 'red')
source = ColumnDataSource(data)

p = figure(plot_width=800, plot_height=300,
           title="Puntos por Nota",
           tools="pan,wheel_zoom,box_zoom,reset")

p.scatter('Nota1','Nota2',
          color="color",
          source=source)

p.add_tools(HoverTool(tooltips=[("Label", "@Label"),
                                ("Nota1", "@Nota1"),
                                ("Nota1", "@Nota2")]))

show(p)

In [97]:
# Codigo basado en 
#
class SVM():
    
    def __init__(self):
        self.w = np.array([1, 2])
    def train(self, X, y):
        self.w = np.random.uniform(size=(1,X.shape[0]))

        self.data = X
        # { ||w||: [w,b] }
        opt_dict = {}

        transforms = [[1,1],
                      [-1,1],
                      [-1,-1],
                      [1,-1]]

        pos_x = []
        neg_x = []
        for i in enumerate(y):
            if i==1:
                pos_x.append(X[i,:])
            else:
                neg_x.append(X[i,:])

        
        self.max_feature_value = max(np.max(X,axis=1))
        self.min_feature_value = min(np.min(X,axis=1))
        all_data = None

        # support vectors yi(xi.w+b) = 1
        

        step_sizes = [self.max_feature_value * 0.1,
                      self.max_feature_value * 0.01,
                      # point of expense:
                      self.max_feature_value * 0.001,]

        
        
        # extremely expensive
        b_range_multiple = 5
        # we dont need to take as small of steps
        # with b as we do w
        b_multiple = 5
        latest_optimum = self.max_feature_value*10

        for step in step_sizes:
            w = np.array([latest_optimum,latest_optimum])
            # we can do this because convex
            optimized = False
            while not optimized:
                for b in np.arange(-1*(self.max_feature_value*b_range_multiple),
                                   self.max_feature_value*b_range_multiple,
                                   step*b_multiple):
                    for transformation in transforms:
                        w_t = w*transformation
                        found_option = True
                        # weakest link in the SVM fundamentally
                        # SMO attempts to fix this a bit
                        # yi(xi.w+b) >= 1
                        # 
                        # #### add a break here later..
                        for i,xi in enumerate(self.data):
                            yi=y[i]
                            if not yi*(np.dot(w_t,xi)+b) >= 1:
                                found_option = False
                                    
                        if found_option:
                            opt_dict[np.linalg.norm(w_t)] = [w_t,b]

                if w[0] < 0:
                    optimized = True
                    print('Optimized a step.')
                else:
                    w = w - step

            norms = sorted([n for n in opt_dict])
            #||w|| : [w,b]
            opt_choice = opt_dict[norms[0]]
            self.w = opt_choice[0]
            self.b = opt_choice[1]
            latest_optimum = opt_choice[0][0]+step*2
            

    def predict(self,features):
        # sign( x.w+b )
        classification = np.sign(np.dot(np.array(features),self.w)+self.b)
        return classification
        

In [106]:
m_SVM = SVM()
m_SVM.train(X,y)

Optimized a step.
Optimized a step.
Optimized a step.


$(X_1*W_1+x_2*w_2)+b) = 1

$

In [101]:
p = figure(plot_width=800, plot_height=300,
           title="Puntos por Nota",
           toolbar_location=None,
           tools="")

p.scatter('Nota1','Nota2',
          color="color",
          source=source)

p.add_tools(HoverTool(tooltips=[("Label", "@Label"),
                                ("Nota1", "@Nota1"),
                                ("Nota1", "@Nota2")]))
def hyperplane(x,w,b,v):
    return (-w[0]*x-b+v) / w[1]

datarange = (m_SVM.min_feature_value*0.9,m_SVM.max_feature_value*1.1)


hyp_x_min = datarange[0]
hyp_x_max = datarange[1]

# (w.x+b) = 1
# positive support vector hyperplane
psv1 = hyperplane(hyp_x_min, m_SVM.w, m_SVM.b, 1.)
psv2 = hyperplane(hyp_x_max, m_SVM.w, m_SVM.b, 1.)
p.line([hyp_x_min,hyp_x_max],[psv1,psv2],color="blue")

psv1 = hyperplane(hyp_x_min, m_SVM.w, m_SVM.b, 0.)
psv2 = hyperplane(hyp_x_max, m_SVM.w, m_SVM.b, 0.)
p.line([hyp_x_min,hyp_x_max],[psv1,psv2],color="orange")


psv1 = hyperplane(hyp_x_min, m_SVM.w, m_SVM.b, -1.)
psv2 = hyperplane(hyp_x_max, m_SVM.w, m_SVM.b, -1.)
p.line([hyp_x_min,hyp_x_max],[psv1,psv2],color="red")


show(p)